In [ ]:
!nvidia-smi

Sat Jul  8 23:26:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1. Clone the github repo

In [ ]:
!git clone https://github.com/Wangt-CN/DisCo

fatal: destination path 'DisCo' already exists and is not an empty directory.



### 2. Install the package

Ps: Most errors are due to the unsuccessful package installation, please check the installation carefully.


In [ ]:
!pip install --user torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --user progressbar psutil pymongo simplejson yacs boto3 pyyaml ete3 easydict deprecated future django orderedset python-magic datasets h5py omegaconf einops ipdb
!pip install --user --exists-action w -r DisCo/requirements.txt
!pip install git+https://github.com/microsoft/azfuse.git

## for acceleration
!pip install --user deepspeed==0.6.3
!pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

### 3. Download the pretrained model
Feel free to use our other [checkpoints](https://github.com/Wangt-CN/DisCo#model-checkpoint-google-cloud-tiktok-training-data-fid-fvd-188--more-tiktok-style-training-data-fid-fvd-157) or change to your own model

In [ ]:
!git clone https://huggingface.co/lambdalabs/sd-image-variations-diffusers
!wget https://storage.googleapis.com/disco-checkpoint-share/checkpoint_ft/moretiktok_nocfg/mp_rank_00_model_states.pt

--2023-07-08 22:44:46--  https://storage.googleapis.com/disco-checkpoint-share/checkpoint_ft/moretiktok_nocfg/mp_rank_00_model_states.pt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 142.250.145.128, 173.194.79.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4547478081 (4.2G) [application/octet-stream]
Saving to: ‘mp_rank_00_model_states.pt’

mp_rank_00_model_st 100%[===================>]   4.23G  32.2MB/s    in 2m 31s  

2023-07-08 22:47:18 (28.7 MB/s) - ‘mp_rank_00_model_states.pt’ saved [4547478081/4547478081]



### 4. Start Running

In [ ]:
import os
os.chdir('/content/DisCo')
os.getcwd()

'/content/DisCo'

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_ENABLE"] = "0"

from utils.wutils_ldm import *
from agent import Agent_LDM, WarmupLinearLR, WarmupLinearConstantLR
import torch
from config import BasicArgs
from utils.lib import *
# from utils.args import parse_with_cf
from utils.dist import dist_init
from dataset.tsv_dataset import make_data_sampler, make_batch_data_sampler
from finetune_sdm_yaml import get_loader_info, make_data_loader
torch.multiprocessing.set_sharing_strategy('file_system')

[2023-07-08 23:59:50 <wutils_ldm.py:150> <module>] <utils.py>: Deep Learning Utils @ Chenfei Wu


WANDB_ENABLE: 0
2023-07-08 23:59:52,674.674 43664:common.py:1785 setup_yaml(): python 3 env


[2023-07-08 23:59:53 <wutils.py:132> <module>] <utils.py>: Deep Learning Utils @ Chenfei Wu
[2023-07-08 23:59:53 <wutils.py:132> <module>] <utils.py>: Deep Learning Utils @ Chenfei Wu
[2023-07-08 23:59:53 <__init__.py:71> BasicArgs] Detected unknown Node f3451ce6be71.
[2023-07-08 23:59:53 <__init__.py:71> BasicArgs] Detected unknown Node f3451ce6be71.


In [ ]:
from utils.args import sharedArgs
manual_args = ['--cf', 'config/ref_attn_clip_combine_controlnet/app_demo_image_edit.py', '--eval_visu', 'True', '--root_dir', '/content/run_test', '--local_train_batch_size', '32', '--local_eval_batch_size', '32', '--log_dir', 'exp/tiktok_ft', '--epochs', '20', '--deepspeed', '--eval_step', '500',
               '--save_step', '500', '--gradient_accumulate_steps', '1', '--learning_rate', '2e-4', '--fix_dist_seed', 'True', '--loss_target',
               'noise', '--unet_unfreeze_type', 'all', '--guidance_scale', '3', '--refer_sdvae', 'True', '--ref_null_caption', 'False', '--combine_clip_local', 'True', '--combine_use_mask', 'True', '--conds', 'poses','masks', '--pretrained_model', '/content/mp_rank_00_model_states.pt', '--pretrained_model_path', '/content/sd-image-variations-diffusers', '--eval_save_filename', 'try']
parsed_args = sharedArgs.parser.parse_args(args=manual_args)

###### process the args #######
if parsed_args.root_dir:
    BasicArgs.root_dir = parsed_args.root_dir
else:
    parsed_args.root_dir = BasicArgs.root_dir
parsed_args.pretrained_model_path = os.path.join(parsed_args.root_dir, parsed_args.pretrained_model_path)

def parse_with_cf(parsed_args):
    """This function will set args based on the input config file.
    (1) it only overwrites unset parameters,
        i.e., these parameters not set from user command line input
    (2) it also sets configs in the config file but declared in the parser
    """
    # convert to EasyDict object,
    # enabling access from attributes even for nested config
    # e.g., args.train_datasets[0].name
    args = edict(vars(parsed_args))
    if os.path.exists(parsed_args.cf):
        cf = import_filename(parsed_args.cf)
        config_args = edict(vars(cf.Args))
        override_keys = {arg[2:].split("=")[0] for arg in manual_args
                         if arg.startswith("--")}
        # import pdb;pdb.set_trace()
        for k, v in config_args.items():
            if k not in override_keys:
                setattr(args, k, v)
    else:
        raise NotImplementedError('Config filename %s does not exist.' % args.cf)
    return args

args = parse_with_cf(parsed_args)

args.n_gpu = T.cuda.device_count() # local size
args.local_size = args.n_gpu
if args.root_dir not in args.log_dir:
    args.log_dir = os.path.join(args.root_dir, args.log_dir)
if args.stepwise_sample_depth == -1:
    args.interpolation = None
    args.interpolate_mode = None
if args.interpolation != "interpolate":
    args.interpolate_mode = None

assert args.eval_step > 0, "eval_step must be positive"
assert args.save_step > 0, "save_step must be positive"

dist_init(args)
args.dist = args.distributed
args.nodes = args.num_nodes
args.world_size = args.num_gpus
args.train_batch_size = args.local_train_batch_size * args.world_size
args.eval_batch_size = args.local_eval_batch_size * args.world_size
#############################################

cf = import_filename(args.cf)
Net, inner_collect_fn = cf.Net, cf.inner_collect_fn

dataset_cf = import_filename(args.dataset_cf)
BaseDataset = dataset_cf.BaseDataset

# args = update_args(parsed_args, args)

# init models
logger.info('Building models...')
model = Net(args)
print(f"Args: {edict(vars(args))}")

[2023-07-08 23:59:59 <<ipython-input-3-ed4eaf604d1c>:69> <cell line: 69>] Building models...
[2023-07-08 23:59:59 <<ipython-input-3-ed4eaf604d1c>:69> <cell line: 69>] Building models...


distributed training ... presumbly debug with 1 GPU
Using seed 42 for rank 0
Using seed 42 for torch.cuda
Loading pre-trained image_encoder from /content/sd-image-variations-diffusers/image_encoder
Loading pre-trained vae from /content/sd-image-variations-diffusers/vae
Loading pre-trained unet from /content/sd-image-variations-diffusers/unet
use sd vae to init the controlnet condition embedding
Args: {'root_dir': '/content/run_test', 'cf': 'config/ref_attn_clip_combine_controlnet/app_demo_image_edit.py', 'pretrained_model': '/content/mp_rank_00_model_states.pt', 'pretrained_model_lora': None, 'pretrained_model_controlnet': None, 'debug': False, 'debug_seed': False, 'debug_dataloader': False, 'log_dir': '/content/run_test/exp/tiktok_ft', 'deepspeed': True, 'use_amp': False, 'seed': 42, 'fix_dist_seed': True, 'tiktok_data_root': 'keli/dataset/TikTok_dataset/', 'img_size': [256, 256], 'max_video_len': 1, 'debug_max_video_len': 1, 'conds': ['poses', 'masks'], 'gradient_checkpointing': True

In [ ]:
logger.warning("Do eval_visu...")
if getattr(args, 'refer_clip_preprocess', None):
    eval_dataset = BaseDataset(args, args.val_yaml, split='val', preprocesser=model.feature_extractor)
else:
    eval_dataset = BaseDataset(args, args.val_yaml, split='val')
eval_dataloader, eval_info = make_data_loader(
    args, args.local_eval_batch_size,
    eval_dataset)


trainer = Agent_LDM(args=args, model=model)
trainer.eval_demo_pre()

[2023-07-09 00:00:49 <<ipython-input-4-a21f1cb1709e>:3> <cell line: 3>] Do eval_visu...
[2023-07-09 00:00:49 <<ipython-input-4-a21f1cb1709e>:3> <cell line: 3>] Do eval_visu...


number of samples: 25
Specify the load model path, not use deepspeed but the pytorch original load func


[2023-07-09 00:01:11 <wutils_ldm.py:456> file2data] Loaded data from /content/mp_rank_00_model_states.pt
[2023-07-09 00:01:11 <wutils_ldm.py:456> file2data] Loaded data from /content/mp_rank_00_model_states.pt
[2023-07-09 00:01:15 <wutils_ldm.py:701> adaptively_load_state_dict] Strictly Loaded state_dict.
[2023-07-09 00:01:15 <wutils_ldm.py:701> adaptively_load_state_dict] Strictly Loaded state_dict.
[2023-07-09 00:01:15 <agent.py:808> load_checkpoint_for_deepspeed_diff_gpu] Loaded checkpoint /content/mp_rank_00_model_states.pt of global_step 7200
[2023-07-09 00:01:15 <agent.py:808> load_checkpoint_for_deepspeed_diff_gpu] Loaded checkpoint /content/mp_rank_00_model_states.pt of global_step 7200


[2023-07-09 00:01:16,138] [INFO] [logging.py:69:log_dist] [Rank 0] DeepSpeed info: version=0.6.3, git-hash=unknown, git-branch=unknown
[2023-07-09 00:01:16,445] [INFO] [engine.py:278:__init__] DeepSpeed Flops Profiler Enabled: False
[2023-07-09 00:01:16,455] [INFO] [config.py:1059:print] DeepSpeedEngine configuration:
[2023-07-09 00:01:16,458] [INFO] [config.py:1063:print]   activation_checkpointing_config  {
    "partition_activations": false, 
    "contiguous_memory_optimization": false, 
    "cpu_checkpointing": false, 
    "number_checkpoints": null, 
    "synchronize_checkpoint_boundary": false, 
    "profile": false
}
[2023-07-09 00:01:16,460] [INFO] [config.py:1063:print]   aio_config ................... {'block_size': 1048576, 'queue_depth': 8, 'thread_count': 1, 'single_submit': False, 'overlap_events': True}
[2023-07-09 00:01:16,462] [INFO] [config.py:1063:print]   amp_enabled .................. False
[2023-07-09 00:01:16,464] [INFO] [config.py:1063:print]   amp_params ......

[2023-07-09 00:01:18 <agent.py:237> prepare_dist_model] Successfully built models with {'trainable': 1581790996, 'frozen': 387620071, 'trainable_fp32': 0, 'trainalbe_fp16': 1581790996, 'frozen_fp32': 303966208, 'frozen_fp16': 83653863} parameters
[2023-07-09 00:01:18 <agent.py:237> prepare_dist_model] Successfully built models with {'trainable': 1581790996, 'frozen': 387620071, 'trainable_fp32': 0, 'trainalbe_fp16': 1581790996, 'frozen_fp32': 303966208, 'frozen_fp16': 83653863} parameters


Mode [all]: There are 686 modules in unet to be set as requires_grad=True.


In [ ]:
def load_image(image):
    if not image.mode == "RGB":
        image = image.convert("RGB")
    return image

@torch.no_grad()
def inference(reference_fg, fg_mask, ref_bg_image, bg_mask, skeleton_img, *args, **kwargs):
    reference_fg = load_image(reference_fg)
    fg_mask = load_image(fg_mask)
    ref_bg_image = load_image(ref_bg_image)
    bg_mask = load_image(bg_mask)
    skeleton_img = load_image(skeleton_img)

    input_data = [reference_fg, fg_mask, ref_bg_image, bg_mask, skeleton_img]
    output_image = trainer.eval_demo_run(input_data, eval_dataset=eval_dataset)
    return output_image

@torch.no_grad()
def inference_masked(reference_fg, ref_bg_image, skeleton_img, *args, **kwargs):
    reference_fg = load_image(reference_fg)
    ref_bg_image = load_image(ref_bg_image)
    skeleton_img = load_image(skeleton_img)

    input_data = [reference_fg, ref_bg_image, skeleton_img]
    output_image = trainer.eval_demo_run_masked(input_data, eval_dataset=eval_dataset)
    return output_image

### 5. Launch the gradio demo

In [ ]:


import gradio as gr
'''
launch app
'''
title = "DisCo Demo (Video Demo Comming Soon!)"
description = """<p style='text-align: center'> <a href='https://disco-dance.github.io/' target='_blank'>Project Page</a> | <a href='https://arxiv.org/pdf/2212.11270.pdf' target='_blank'>Paper</a> | <a href='https://github.com/microsoft/X-Decoder' target='_blank'>Github Repo</a> | <a href='https://youtu.be/wYp6vmyolqE' target='_blank'>Video</a> </p>
<p>Skip the queue by duplicating this space and upgrading to GPU in settings</p>
<a href="https://huggingface.co/spaces/xdecoder/Demo?duplicate=true"><img src="https://bit.ly/3gLdBN6" alt="Duplicate Space"></a>
"""



with gr.Blocks() as demo:
    gr.Markdown(
    """
    # DisCo Demo (Video Demo Comming Soon!)
    Start edit the human with provided human foreground, background, pose.

    Note that for self-uploaded images, TikTok-Style human images are preferred.

    [Project Page](https://disco-dance.github.io/) | [Github](https://github.com/Wangt-CN/DisCo)
    """)

    with gr.Row().style(equal_height=False):
        with gr.Column(min_width=400, scale=2):
            input_fg = gr.Image(type='pil',label="Foreground Image")
            gr.Examples(examples=["./demo_data/fg/masked_images/00035.png", "./demo_data/fg/masked_images/00335.png", "./demo_data/fg/masked_images/00147.png", "./demo_data/fg/masked_images/00072.png", "./demo_data/fg/masked_images/00115.png"], inputs=input_fg)

            input_bg = gr.Image(type='pil',label="Background Image")
            gr.Examples(examples=["./demo_data/bg/masked_images/00035.png", "./demo_data/bg/masked_images/00335.png", "./demo_data/bg/masked_images/00147.png", "./demo_data/bg/masked_images/00072.png", "./demo_data/bg/masked_images/00115.png"], inputs=input_bg)

            input_pose = gr.Image(type='pil',label="Target Pose",scale=1)
            gr.Examples(examples=["./demo_data/pose_img/0049.png","./demo_data/pose_img/0198.png","./demo_data/pose_img/0213.png","./demo_data/pose_img/0264.png","./demo_data/pose_img/0144.png","./demo_data/pose_img/0054.png"], inputs=input_pose)

            btn = gr.Button("Generate")


        with gr.Column(min_width=150):
            output_img = gr.Image(type='pil',label="Edited Human Image")

    btn.click(inference_masked, inputs=[input_fg, input_bg, input_pose], outputs=[output_img])

demo.queue(concurrency_count=2)
demo.launch(share=True)

<ipython-input-7-11e6eef69ecf>:31: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  with gr.Row().style(equal_height=False):


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e249d08ffc1f80b5e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
